#  Data Analysis and Results Collection

This file is for automaticly output all the analysis for all dataset at once 

In [ ]:
#!pip install xlwt

In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
#Imports
import math
import librosa
import librosa.display
import IPython.display as ipd

import pyAudioAnalysis
import pydub
import ffmpeg
import os
import sklearn

from pydub import AudioSegment
from pydub.silence import split_on_silence

import numpy
import numpy as np
import matplotlib.pyplot as plt

import xlwt
from xlwt import Workbook

from numpy import sin, cos, pi, linspace, arange, log10, absolute
from numpy.random import randn
from scipy.signal import lfilter, lfilter_zi, filtfilt, butter, freqz, welch
from scipy.io.wavfile import read , write
from scipy.signal import spectrogram
from matplotlib import pyplot
from matplotlib.pyplot import ylabel, xlabel
from matplotlib.pyplot import plot, legend, show, grid, figure, savefig#, hold

In [3]:
#List of Words
words = ["_chata_0","_chapa_0","_chave_0","_lata_0","_lapa_0","_lava_0","_casa_0","_capa_0","_cave_0","_cata_0","_chuta_0","_chupa_0","_chuva_0","_farta_0","_farpa_0","_farda_0","_ripa_0","_rita_0","_rica_0"]

#List of Dataset versions
d = {"one": 1, "two": 2}
iterable = d.keys()

#List of Phonemes 
phoneme = ["_0", "_1"]

#List of people
p = ['A','J','N']


In [4]:
segments = []
errors = 0

Process_Select = False # False for audio save / True for picture save
purpose = True # False for Pitch Segmentation / True for Word Segmentation

for y in words:
    for item in iterable:      
        for x in p:
            for t in phoneme:

                # Write file path
                l = str(d[item])
                os.chdir(r'C:\Users\Dasil\1. Processamento Digital do Sinal\Project')
                #MFCC Analysis
                filename = 'audio/WD_data/'+x+y+l+t+'.wav' #for Test DataSet
                signal, fs = librosa.load(filename)
                
                # Some Audio Errors, the solution maybe in here 
                # https://stackoverflow.com/questions/68130038/valueerror-input-signal-length-2-is-too-small-to-resample-from-44100-16000
                
                sr = fs
                
                #signal.shape
                mfccs = librosa.feature.mfcc(signal, sr=fs, n_mfcc=40, n_fft = 1024, hop_length = 50, n_mels = 130,  fmin = 10, fmax = 4000)         

                #Normalyse Y scale
                mfccs = sklearn.preprocessing.scale(mfccs, axis=1)

                #Visualie MFCC 
                plt.figure(figsize=(14,4))
                #librosa.display.specshow(mfccs[1:][10:],x_axis='time',sr=sr)
                librosa.display.specshow(mfccs[1:][:],x_axis='time',sr=sr)
                plt.colorbar(format="%+2f")
                plt.plot()

                plt.savefig('Word Detection/MFCCS_ID/'+x+y+l+t+'.jpeg')
                  
                #Clean the graphics
                plt.close()
                plt.cla()
                plt.clf()
                    

ValueError: Input signal length=0 is too small to resample from 44100->22050

<Figure size 432x288 with 0 Axes>